In [72]:
class Argument:
    def __init__(self, value, mode):
        self.value = value
        self.mode = mode
    
    def __str__(self):
        return str(self.value) + ' ' + str(self.mode)
    
class IntcodeComputer:
    
    def code_parser(self, bigcode):
        bigcode = str(bigcode)
        bigcode = '0'*(5 - len(bigcode)) + bigcode
        if self.verbose:
            print('Bigcode:', bigcode)
        self.opcode = int(bigcode[-2:])
        self.args['1'] = int(bigcode[2])
        self.args['2'] = int(bigcode[1])
        self.args['3'] = int(bigcode[0])
        
    def get_value(self, arg):
        if arg.mode:
            return arg.value
        else:
            return self.intcode[arg.value]
        
    def get_args(self, num_args):
        for i in range(1, num_args+1):
            self.args[str(i)] = Argument(self.intcode[self.pointer + i], self.args[str(i)])
        return [self.get_value(self.args[str(i)]) for i in range(1, num_args+1)]
        
    def add(self):
        arg1, arg2, arg3 = self.get_args(3)
        arg3 = self.args['3'].value
        ans = arg1 + arg2
        self.intcode[arg3] = ans
        self.pointer += 4
        
        if self.verbose:
            print('Add:', arg1, '+', arg2, '=', ans)
            print('To address:', arg3)
            
    def mult(self):
        arg1, arg2, arg3 = self.get_args(3)
        arg3 = self.args['3'].value
        ans = arg1 * arg2
        self.intcode[arg3] = ans
        self.pointer += 4
        
        if self.verbose:
            print('Mult:', arg1, '*', arg2, '=', ans)
            print('To address:', arg3)
        
    def save(self):
        arg1, = self.get_args(1)
        arg1 = self.args['1'].value
        self.pointer += 2
        
        if self.verbose:
            print('User Input:')
        if self.interactive:
            user_input = input()
        else:
            user_input = self.inputs[self.inpointer]
            if self.verbose:
                print(self.inputs[self.inpointer])
            self.inpointer += 1
        if self.verbose:
            print('To address:', arg1)
        self.intcode[arg1] = int(user_input)
        if self.verbose:
            print('Value:', user_input)
        
    def load(self):
        arg1, = self.get_args(1)
        arg1 = self.args['1'].value
        ans = self.intcode[arg1]
        self.outputs.append(ans)
        self.pointer += 2
        
        if self.verbose:
            print('Loading', ans, 'from', arg1)
        
        
    def halt(self):
        print('Intcode Computer Halting')
        self.halted = True
        print('Last output:')
        try:
            print(self.outputs[-1])
        except:
            pass

    def jump_if_true(self):
        arg1, arg2 = self.get_args(2)
        if arg1 != 0:
            self.pointer = arg2
        else:
            self.pointer += 3        
        
    def jump_if_false(self):
        arg1, arg2 = self.get_args(2)
        if not(arg1):
            self.pointer = arg2
        else:
            self.pointer += 3
            
    def less_than(self):
        arg1, arg2, arg3 = self.get_args(3)
        arg3 = self.args['3'].value        
        self.intcode[arg3] = int(arg1 < arg2)
        self.pointer += 4
    
    def equals(self):
        arg1, arg2, arg3 = self.get_args(3)
        arg3 = self.args['3'].value        
        self.intcode[arg3] = int(arg1 == arg2)
        self.pointer += 4
        
    def __init__(self):
        self.inpointer = 0
        self.pointer = 0
        self.intcode = []
        self.opcode = None
        self.args = dict()
        self.halted = False
        self.outputs = []
        self.inputs = []
        self.verbose = False
        self.interactive = True
        self.operation = {
            '1': self.add,
            '2': self.mult,
            '3': self.save,
            '4': self.load,
            '5': self.jump_if_true,
            '6': self.jump_if_false,
            '7': self.less_than,
            '8': self.equals,
            '99': self.halt
        }
        
    def set_intcode(self, intcode):
        self.intcode = intcode
        
    def set_inputs(self, inputs):
        self.inputs = inputs
        self.interactive = False
    
    def set_verbose(self, verbose):
        self.verbose = verbose
    
    def print_state(self):
        for key, value in glados.__dict__.items():
            if str(key) not in ['intcode', 'operation']:
                if type(value) == dict:
                    print(str(key) + ':')
                    for key2, value2 in value.items():
                        print('\t' + str(key2) + ':', value2)
                else:
                    print(str(key) + ':', value)
                 
    def read_step(self):
        self.code_parser(self.intcode[self.pointer])
        self.operation[str(self.opcode)]()
        if self.verbose:
            self.print_state()
            print('\n' + '='*10 + '\n')

In [73]:
input_file = open('input')
intcode = [int(char) for char in input_file.read().split(',')]
input_file.close()

In [74]:
glados = IntcodeComputer()
glados.set_intcode(intcode)

In [75]:
while glados.halted == False:
    glados.set_verbose(True)
    glados.read_step()

Bigcode: 00003
User Input:
5
To address: 225
Value: 5
inpointer: 0
pointer: 2
opcode: 3
args:
	1: 225 0
	2: 0
	3: 0
halted: False
outputs: []
inputs: []
verbose: True
interactive: True


Bigcode: 00001
Add: 5 + 1100 = 1105
To address: 6
inpointer: 0
pointer: 6
opcode: 1
args:
	1: 225 0
	2: 6 0
	3: 6 0
halted: False
outputs: []
inputs: []
verbose: True
interactive: True


Bigcode: 01105
inpointer: 0
pointer: 238
opcode: 5
args:
	1: 1 1
	2: 238 1
	3: 0
halted: False
outputs: []
inputs: []
verbose: True
interactive: True


Bigcode: 01105
inpointer: 0
pointer: 241
opcode: 5
args:
	1: 0 1
	2: 99999 1
	3: 0
halted: False
outputs: []
inputs: []
verbose: True
interactive: True


Bigcode: 01105
inpointer: 0
pointer: 247
opcode: 5
args:
	1: 227 1
	2: 247 1
	3: 0
halted: False
outputs: []
inputs: []
verbose: True
interactive: True


Bigcode: 01005
inpointer: 0
pointer: 250
opcode: 5
args:
	1: 227 0
	2: 99999 1
	3: 0
halted: False
outputs: []
inputs: []
verbose: True
interactive: True


Bigcode: 0

interactive: True


Bigcode: 01005
inpointer: 0
pointer: 565
opcode: 5
args:
	1: 224 0
	2: 569 1
	3: 0
halted: False
outputs: []
inputs: []
verbose: True
interactive: True


Bigcode: 00101
Add: 1 + 68788 = 68789
To address: 223
inpointer: 0
pointer: 569
opcode: 1
args:
	1: 1 1
	2: 223 0
	3: 223 0
halted: False
outputs: []
inputs: []
verbose: True
interactive: True


Bigcode: 00108
inpointer: 0
pointer: 573
opcode: 8
args:
	1: 677 1
	2: 677 0
	3: 224 0
halted: False
outputs: []
inputs: []
verbose: True
interactive: True


Bigcode: 00102
Mult: 2 * 68789 = 137578
To address: 223
inpointer: 0
pointer: 577
opcode: 2
args:
	1: 2 1
	2: 223 0
	3: 223 0
halted: False
outputs: []
inputs: []
verbose: True
interactive: True


Bigcode: 01005
inpointer: 0
pointer: 580
opcode: 5
args:
	1: 224 0
	2: 584 1
	3: 0
halted: False
outputs: []
inputs: []
verbose: True
interactive: True


Bigcode: 00101
Add: 1 + 137578 = 137579
To address: 223
inpointer: 0
pointer: 584
opcode: 1
args:
	1: 1 1
	2: 223 0
	3: 22